In [39]:
# Part1---This notebook gives outline of the scraping method
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn

In [40]:
# to keep a real time record of the scraping process 
# As imdb was giving record for 10000 movies per run, I had to run multiple times this script with changes in the years
import logging

logging.basicConfig(level=logging.DEBUG)
# various lists in which the data from each movie container (which is a container containing info about individual movies) is collected 
names=[]
years=[]
grades=[]
durations=[]
genres=[]
ratings=[]
votes=[]
metascores=[]
directors = []
stars = []
income=[]

start_time = time()
request = 0
#full url -->
#https://www.imdb.com/search/title?title_type=feature&release_date=2018-01-01,2018-12-31&languages=en
#https://www.imdb.com/search/title?title_type=feature&release_date=2017-01-01,2017-12-31&languages=en
#https://www.imdb.com/search/title?title_type=feature&release_date=2016-01-01,2016-12-31&languages=en
#https://www.imdb.com/search/title?title_type=feature&release_date=2015-01-01,2015-12-31&languages=en
#https://www.imdb.com/search/title?title_type=feature&release_date=2013-01-01,2014-12-31&languages=en
#https://www.imdb.com/search/title?title_type=feature&release_date=2011-01-01,2012-12-31&languages=en
#https://www.imdb.com/search/title?title_type=feature&release_date=2009-01-01,2010-12-31&languages=en
params = {
    'title_type' : 'feature',
    'release_date' : '2009-01-01,2010-12-31',
    'languages' : 'en',
    'start' : '1',
    'ref' : 'adv_nxt'
}

url = 'https://www.imdb.com/search/title'

i = 1
# already know that the number of movies under the selected criteria is less than 10000
while i<10000:
    params['start'] = str(i)
    # sleep is added to avoid continuous bombarding the requests to imdb
    sleep(randint(8,15))
    # resp will generate specific url for each request combining url and params
    resp = requests.get(url, params=params)
    
    page_html = BeautifulSoup(resp.text, 'html.parser')
    # as each page has 50 movie results, thus there are 50 movie containers per page (except the last page), findAll eturns all the matches 
    movie_containers = page_html.findAll('div', class_='lister-item mode-advanced')
    
    if not movie_containers:
        break;
    #this will loop through the 50 movie containers obtained in the previous step    
    for movie in movie_containers:
        # we get respective info about each moving by going inside the tags and then appending the info to the lists made above
        
        #scraping the movie's name
        name=movie.h3.a.text
        names.append(name)
        
        #scraping the year of movie's release  
        year=movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
        years.append(year)
        
        #scraping the movie's certificate
        grade=movie.p.find('span', class_ = 'certificate')
        grades.append(grade and grade.text)
        
        #scraping the movie's length
        duration=movie.p.find('span', class_ = 'runtime')
        durations.append(duration and duration.text)
        
        #scraping the movie's genre
        genre=movie.p.find('span', class_ = 'genre')
        genres.append(genre and genre.text)
        
        #scraping the movie's rating
        rating=movie.strong
        ratings.append(rating and rating.text)
            
        #scraping the number of votes recieved by the movie
        vote1=movie.find('span',attrs={'name':'nv'})
        votes.append(vote1 and vote1['data-value'])
        
        #scraping the movie's directors and stars
        directors_and_stars = movie.select_one('p:nth-of-type(3)')
        directors_and_stars = directors_and_stars and directors_and_stars.text
        try:
            if directors_and_stars is None:
                ds, ss= None, None
            else: 
                ds, ss = directors_and_stars.split('|')
        except:
            if directors_and_stars.strip().startswith('Director'):
                ds, ss = directors_and_stars, None
            elif directors_and_stars.strip().startswith('Star'):
                ds, ss = None, directors_and_stars
            else:
                ds, ss = None, None
            
        directors.append(ds and ds.split(':')[1].strip().replace('\n', ''))
        stars.append(ss and ss.split(':')[1].strip().replace('\n', ''))
        
        #scraping the revenue collected by the movie'#scraping the movie's rating
        gross=movie.select_one('p:nth-of-type(4)')
        if gross is None:
            gross1=None
        else:
            gross = gross.select_one('span:nth-of-type(5)')
            if gross is None:
                gross1 = None
            else:
                gross1 = gross['data-value'] 
        income.append(gross1)
        
        #scraping the movie's metascore
        if movie.find('div', class_ = 'ratings-metascore') is not None:
            movie_metascore=movie.findAll('span', {'class':['metascore']})
            for score in movie_metascore:
                metascores.append(score and score.text)
        else:
            metascores.append('NA')
    i += 50

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=1&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=51&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=101&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&star

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=1601&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=1651&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=1701&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=e

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=3201&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=3251&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=3301&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=e

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=4801&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=4851&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=4901&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=e

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=6401&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=6451&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=en&start=6501&ref=adv_nxt HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.imdb.com
DEBUG:urllib3.connectionpool:https://www.imdb.com:443 "GET /search/title?title_type=feature&release_date=2009-01-01%2C2010-12-31&languages=e

In [42]:
# building the scraped dats's dataframe
movies_2009_10=pd.DataFrame({'movie':names,'year':years,'certificate':grades,'duration':durations, 'genre':genres,'rating':ratings,'vote':votes,'metascore':metascores, 'directors':directors, 'stars':stars, 'gross':income})

print(movies_2009_10.info())
movies_2011_12.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7666 entries, 0 to 7665
Data columns (total 11 columns):
certificate    3045 non-null object
directors      7506 non-null object
duration       6621 non-null object
genre          7428 non-null object
gross          700 non-null object
metascore      7666 non-null object
movie          7666 non-null object
rating         5346 non-null object
stars          7415 non-null object
vote           5349 non-null object
year           7666 non-null object
dtypes: object(11)
memory usage: 658.9+ KB
None


,certificate,directors,duration,genre,gross,metascore,movie,rating,stars,vote,year
0,PG-13,Garry Marshall,113 min,"\nComedy, Romance","54,544,638",22,New Year's Eve,5.7,"Sarah Jessica Parker, Jessica Biel, Ashton Kut...",75532,(2011)
1,PG-13,Joss Whedon,143 min,"\nAction, Adventure, Sci-Fi","623,357,910",69,The Avengers,8.1,"Robert Downey Jr., Chris Evans, Scarlett Johan...",1142465,(2012)
2,PG-13,David Yates,130 min,"\nAdventure, Drama, Fantasy","381,011,219",87,Harry Potter and the Deathly Hallows: Part 2,8.1,"Daniel Radcliffe, Emma Watson, Rupert Grint, M...",672763,(2011)
3,PG-13,Tom Hooper,158 min,"\nDrama, History, Musical","148,809,770",63,Les Misérables,7.6,"Hugh Jackman, Russell Crowe, Anne Hathaway, Am...",280169,(2012)
4,PG-13,Christopher Nolan,164 min,"\nAction, Thriller","448,139,099",78,The Dark Knight Rises,8.4,"Christian Bale, Tom Hardy, Anne Hathaway, Gary...",1352837,(2012)


In [43]:
movies_2009_10.to_csv('movies_2009_10.csv')